# Housing Prices Prediction

## Initialize Fiddler Client
We begin this section as usual by establishing a connection to our
Fiddler instance. We can establish this connection either by specifying 
our credentials directly, or by utilizing our `fiddler.ini` file. More
information can be found in the [setup](https://github.com/fiddler-labs/fiddler-samples/blob/master/content_root/tutorial/00%20Setup.ipynb) section.

In [ ]:
import fiddler as fdl

# client = fdl.FiddlerApi(url=url, org_id=org_id, auth_token=auth_token)
client = fdl.FiddlerApi()

## Load baseline
Here we will load in our baseline dataset from a csv called `housing-prices.csv`. We will
also create a schema using this information.

In [ ]:
import pandas as pd
baseline_df = pd.read_csv('/app/fiddler_samples/samples/datasets/housing/housing-prices.csv')
baseline_schema = fdl.DatasetInfo.from_dataframe(baseline_df, max_inferred_cardinality=1000)


In [ ]:
baseline_df

## Setup Monitoring
Now, we will setup a project, and use Fiddler's surrogate model capability to generate a model. 
Projects are one of the key entities of Fiddler. They are convenient containers 
for housing the models and datasets associated with a given ML use case. Specifics about both
projects and models can be found [here](https://docs.fiddler.ai/components/).

In [ ]:
project_id = 'tutorial'
name = 'housing_prices'
target='SalePrice'
input_df = baseline_df.drop(columns=['Id', target])
features = list(input_df.columns)

In [ ]:
## setup/cleanup project
if project_id in client.list_projects():
    client.delete_model(project_id, name)
    client.delete_dataset(name)
else:
    client.create_project(project_id)


client.create_surrogate_model(
    project_id,
    name,
    baseline_df,
    baseline_schema,
    target,
    features
)

## Send Monitoring Events
In this step, we will be simulating traffic to send for our model monitoring by using 
[publish_event](https://docs.fiddler.ai/api-reference/python-package/#publish-event). 
This will be the equivalent of running our model separately on data, and either 
sending to Fiddler then, or saving this information to a log and sending at a later point.

For the purposes of this demonstration, we will be going with a log approach simulated
by using our original dataframe.  To most accurately simulate this as a time-series event,
we will also be calling  a function to generate a timestamp in the last two weeks. Real 
data will ideally  have a timestamp related to when the event took place; otherwise, 
the current time will be used.

**Note**: The timestamp must be in UTC milliseconds. See 
[here](https://docs.fiddler.ai/api-reference/python-package/#publish-event) for more details

In [ ]:
import datetime
import time
from IPython.display import clear_output
from random import sample, randint
NUM_EVENTS_TO_SEND = 50

def getTimestampFromPastTwoWeeks():
    """
    Generate a randomized timestamp from the past two weeks. Timestamp is in 
    milliseconds since epoch in UTC.
    """
    TWO_WEEKS_MS = 604800 * 2 * 1000
    current_time_in_ms = round(time.time() * 1000)
    
    random_time_in_past_two_weeks = current_time_in_ms - randint(0, TWO_WEEKS_MS)
    return random_time_in_past_two_weeks

# Convert this dataframe into a list of dictionary events, where each event is its own dictionary
event_list_dict = input_df.sample(n=NUM_EVENTS_TO_SEND, random_state=42).to_dict(orient='records') 

for ind, event_dict in enumerate(event_list_dict):
    event_ms_time_stamp = getTimestampFromPastTwoWeeks()
    result = client.publish_event(project_id, model_id, event_dict, event_time_stamp=event_ms_time_stamp)
    
    clear_output(wait = True)
    readable_timestamp = datetime.datetime.fromtimestamp(event_ms_time_stamp/1000.0)
    
    print(f'Sending {ind+1} / {NUM_EVENTS_TO_SEND} \n{readable_timestamp} UTC: \n{event_dict}')
    time.sleep(1)